# Group Assignment PBD PL1 - Multinomial Logistic Regression & Decision Tree Classifier
## Book 3 of 3: Decision Tree Classifier


Authors:

Catarina Castanheira Nº 92478

João Martins Nº 93259

Joel Paula Nº 93392



Imports we will need for our work.

In [1]:
from pyspark.ml.feature import VectorAssembler, OneHotEncoder, StringIndexer
from pyspark.ml.classification import DecisionTreeClassifier
# from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml import Pipeline

Functions we will use for every model.

In [2]:
import pandas as pd
def showPerformanceOnTest(df_pred, default_prop, ln_count=0):
    if ln_count == 0:
        ln_count = df_pred.count()
    df_tmp = df_pred.filter('prediction == collision_severity_index')
    acc = df_tmp.count() 
    acc = acc / ln_count
    print("Accuracy:", acc)
    
#    #Confusion Matrix /!\ crashes Spark when large dataset!
#    preds_and_labels = df_preds.select(['prediction','collision_severity_index']).withColumnRenamed('collision_severity_index','label').orderBy('prediction')
#    metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))
#    conf_matrix = metrics.confusionMatrix().toArray()
#    acc = sum(conf_matrix.diagonal())/conf_matrix.sum()
#    print("Accuracy:", acc)
#    conf_matrix = pd.DataFrame(conf_matrix)
#    print("Confusion matrix:\n", conf_matrix)

    # Huberty Index
    hix = (acc - default_prop)/(1 - default_prop)
    print("Huberty Index:", hix)
    return acc


In [3]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True) # pretty print tables

Read small dataset from parquet file.

In [4]:
df = spark.read.format("parquet").load("collisions_clean")

In [5]:
df.printSchema()

root
 |-- jurisdiction: string (nullable = true)
 |-- chp_shift: string (nullable = true)
 |-- population: string (nullable = true)
 |-- county_city_location: string (nullable = true)
 |-- special_condition: string (nullable = true)
 |-- beat_type: string (nullable = true)
 |-- chp_beat_type: string (nullable = true)
 |-- city_division_lapd: string (nullable = true)
 |-- chp_beat_class: string (nullable = true)
 |-- beat_number: string (nullable = true)
 |-- primary_road: string (nullable = true)
 |-- secondary_road: string (nullable = true)
 |-- distance: double (nullable = true)
 |-- direction: string (nullable = true)
 |-- intersection: long (nullable = true)
 |-- weather_1: string (nullable = true)
 |-- weather_2: string (nullable = true)
 |-- state_highway_indicator: long (nullable = true)
 |-- caltrans_county: string (nullable = true)
 |-- caltrans_district: string (nullable = true)
 |-- state_route: string (nullable = true)
 |-- route_suffix: string (nullable = true)
 |-- postmi

In [6]:
df.limit(30)

jurisdiction,chp_shift,population,county_city_location,special_condition,beat_type,chp_beat_type,city_division_lapd,chp_beat_class,beat_number,primary_road,secondary_road,distance,direction,intersection,weather_1,weather_2,state_highway_indicator,caltrans_county,caltrans_district,state_route,route_suffix,postmile_prefix,postmile,location_type,ramp_intersection,side_of_highway,tow_away,collision_severity,killed_victims,injured_victims,party_count,primary_collision_factor,pcf_violation_code,pcf_violation_category,pcf_violation,pcf_violation_subsection,hit_and_run,type_of_collision,motor_vehicle_involved_with,pedestrian_action,road_surface,road_condition_1,road_condition_2,lighting,control_device,chp_road_type,pedestrian_collision,bicycle_collision,motorcycle_collision,truck_collision,alcohol_involved,statewide_vehicle_type_at_fault,chp_vehicle_type_at_fault,severe_injury_count,other_visible_injury_count,complaint_of_pain_injury_count,pedestrian_killed_count,pedestrian_injured_count,bicyclist_killed_count,bicyclist_injured_count,motorcyclist_killed_count,motorcyclist_injured_count,collision_date,collision_time
9495,1,9,1600,0,1,interstate,Not Stated,chp other,004,RT 5,RT 41,2640.0,south,0,cloudy,Not Stated,1,KIN,6,5,Not Stated,Not Stated,16.07,highway,Not Stated,northbound,1,property damage only,0,0,2,vehicle code viol...,Not Stated,improper turning,22107,Not Stated,not hit and run,sideswipe,other motor vehicle,no pedestrian inv...,dry,normal,Not Stated,daylight,none,1,0,0,0,1,0,passenger car,07,0,0,0,0,0,0,0,0,0,2008-01-07,11:04:00
9265,1,9,3900,1,3,county road area,Not Stated,chp other,010,KAISER RD,MARIPOSA RD,1584.0,north,0,clear,Not Stated,0,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,0,property damage only,0,0,1,other than driver,Not Stated,other than driver...,Not Stated,Not Stated,not hit and run,hit object,other object,no pedestrian inv...,wet,other,Not Stated,dark with no stre...,none,0,0,0,0,0,0,Not Stated,Not Stated,0,0,0,0,0,0,0,0,0,2008-01-07,06:50:00
4807,5,6,4807,0,0,not chp,Not Stated,not chp,006,MISSISSIPPI ST,SACRAMENTO ST,114.0,east,0,clear,Not Stated,0,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,0,property damage only,0,0,2,vehicle code viol...,Not Stated,dui,23152,A,not hit and run,sideswipe,parked motor vehicle,no pedestrian inv...,dry,normal,Not Stated,dark with street ...,none,0,0,0,0,0,0,Not Stated,Not Stated,0,0,0,0,0,0,0,0,0,2008-01-15,23:30:00
9725,2,9,4300,0,1,state route,Not Stated,chp primary,160,RT 152,FERGUSON RD,280.0,west,0,Not Stated,Not Stated,1,SCL,4,152,Not Stated,Not Stated,12.76,highway,Not Stated,westbound,0,property damage only,0,0,2,vehicle code viol...,Not Stated,improper turning,22107,Not Stated,not hit and run,broadside,other motor vehicle,no pedestrian inv...,Not Stated,Not Stated,Not Stated,Not Stated,functioning,0,0,0,0,1,0,Not Stated,Not Stated,0,0,0,0,0,0,0,0,0,2008-01-04,14:45:00
3000,5,4,3021,0,0,not chp,Not Stated,not chp,023,BEACH BL,CHAPMAN,0.0,Not Stated,1,clear,Not Stated,1,ORA,12,39,Not Stated,Not Stated,9.671,intersection,5,northbound,1,pain,0,2,2,vehicle code viol...,Not Stated,traffic signals a...,21453,A,not hit and run,Not Stated,other motor vehicle,no pedestrian inv...,dry,normal,Not Stated,daylight,functioning,0,0,0,0,0,0,passenger car,01,0,0,2,0,0,0,0,0,0,2008-01-01,12:40:00
9450,1,9,2000,0,1,state route,Not Stated,chp other,012,RT 99,AVENUE 24 1/2,550.0,north,0,cloudy,Not Stated,1,MAD,6,99,Not Stated,Not Stated,24.53,highway,Not Stated,northbound,1,property damage only,0,0,1,other than driver,Not Stated,other than driver...,Not Stated,Not Stated,not hit and run,hit object,other object,no pedestrian inv...,wet,normal,Not Stated,daylight,none,1,0,0,0,0,0,Not Stated,Not Stated,0,0,0,0,0,0,0,0,0,2008-01-04,09:21:00
9580,1,7,1942,0,1,us highway,Not Stated,chp other,102,RT 101,HASKELL AV,215.0,north,0,clear,Not Stated,1,LA,7,101,Not Stated,Not Stated,17.535,r

Split train and test sets.

In [7]:
df_test, df_train = df.randomSplit([0.25,0.75], seed=42)
[df_train.count(), df_test.count()]

[75790, 25418]

In [8]:
df_train.cache()
df_test.cache()

jurisdiction,chp_shift,population,county_city_location,special_condition,beat_type,chp_beat_type,city_division_lapd,chp_beat_class,beat_number,primary_road,secondary_road,distance,direction,intersection,weather_1,weather_2,state_highway_indicator,caltrans_county,caltrans_district,state_route,route_suffix,postmile_prefix,postmile,location_type,ramp_intersection,side_of_highway,tow_away,collision_severity,killed_victims,injured_victims,party_count,primary_collision_factor,pcf_violation_code,pcf_violation_category,pcf_violation,pcf_violation_subsection,hit_and_run,type_of_collision,motor_vehicle_involved_with,pedestrian_action,road_surface,road_condition_1,road_condition_2,lighting,control_device,chp_road_type,pedestrian_collision,bicycle_collision,motorcycle_collision,truck_collision,alcohol_involved,statewide_vehicle_type_at_fault,chp_vehicle_type_at_fault,severe_injury_count,other_visible_injury_count,complaint_of_pain_injury_count,pedestrian_killed_count,pedestrian_injured_count,bicyclist_killed_count,bicyclist_injured_count,motorcyclist_killed_count,motorcyclist_injured_count,collision_date,collision_time
100,5,4,0198,0,0,not chp,Not Stated,not chp,2074,DUBLIN BL,CAMP PARKS BL,0.0,Not Stated,1,clear,Not Stated,0,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,0,pain,0,1,2,vehicle code viol...,Not Stated,automobile right ...,21453,C,not hit and run,broadside,bicycle,no pedestrian inv...,dry,normal,Not Stated,daylight,functioning,0,0,1,0,0,0,Not Stated,Not Stated,0,0,1,0,0,0,1,0,0,2004-06-08,16:25:00
100,5,4,0198,0,0,not chp,Not Stated,not chp,DPS,BLOOMINGTON WY,BLOOMINGTON WY 11478,0.0,Not Stated,2,clear,Not Stated,0,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,0,severe injury,0,1,2,vehicle code viol...,Not Stated,improper turning,22103,Not Stated,not hit and run,broadside,other motor vehicle,no pedestrian inv...,dry,normal,Not Stated,daylight,none,0,0,0,1,0,0,passenger car,Not Stated,1,0,0,0,0,0,0,0,1,2003-05-11,11:30:00
100,5,4,0198,0,0,not chp,Not Stated,not chp,DUBLIN,DUBLIN BL,AMADOR PLAZA RD,400.0,east,0,clear,Not Stated,0,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,0,property damage only,0,0,2,unknown,Not Stated,unknown,Not Stated,Not Stated,misdemeanor,rear end,other motor vehicle,no pedestrian inv...,dry,normal,Not Stated,daylight,none,0,0,0,0,0,0,Not Stated,Not Stated,0,0,0,0,0,0,0,0,0,2007-10-18,14:10:00
100,5,4,0198,0,0,not chp,Not Stated,not chp,NOVALU,DUBLIN BL,INSPIRATION DR,5.0,west,0,clear,Not Stated,0,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,1,property damage only,0,0,2,vehicle code viol...,Not Stated,speeding,22350,Not Stated,not hit and run,rear end,other motor vehicle,no pedestrian inv...,dry,normal,Not Stated,daylight,functioning,0,0,0,0,0,0,passenger car,01,0,0,0,0,0,0,0,0,0,2019-03-17,07:44:00
100,5,4,0198,0,0,not chp,Not Stated,not chp,Not Stated,AMADOR PLAZA BL,DUBLIN BL,245.0,north,0,cloudy,Not Stated,0,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,1,property damage only,0,0,2,vehicle code viol...,Not Stated,automobile right ...,21804,A,not hit and run,broadside,other motor vehicle,no pedestrian inv...,dry,Not Stated,Not Stated,daylight,none,0,0,0,0,0,0,passenger car wit...,01,0,0,0,0,0,0,0,0,0,2012-03-25,13:33:00
100,5,4,0198,0,0,not chp,Not Stated,not chp,Not Stated,BETLEN DR,PROW WY,32.0,south,0,clear,Not Stated,0,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,Not Stated,1,property damage only,0,0,1,vehicle code viol...,Not Stated,improper turning,22107,Not Stated,not hit and run,hit object,fixed object,no pedestrian inv...,dry,normal,Not Stated,daylight,none,0,0,0,0,0,0,passenger car,01,0,0,0,0,0,0,0,0,0,2016-08-02,10:58:00
100,5,4,0198,0,0,not chp,Not Stated,not chp,Not Stated,DOUGHERTY R

Vectorize all features. But for categorical features, we start by indexing them.

In [9]:

target_var = "collision_severity"
target_var_index = target_var+"_index"

# categorical feature with too many different values
exclude_categories = ["primary_road", "secondary_road", "collision_date", "postmile", 
                      "beat_number", "collision_time","county_city_location", 
                      "jurisdiction", "state_route", "pcf_violation"]

cols = df.columns
if target_var in cols:
    cols.remove(target_var)

col_names_categorical = [col for (col, dataType) in df.dtypes if dataType == 'string' and col != target_var and col not in exclude_categories]
    
indexOutputCols = [col+"_index" for col in col_names_categorical]
stringIndexer = StringIndexer(inputCols=col_names_categorical+[target_var], outputCols=indexOutputCols+[target_var_index], handleInvalid="skip")

col_names_numerical = [col for col in cols if col not in col_names_categorical and col not in exclude_categories] 
assembler = VectorAssembler(inputCols=indexOutputCols+col_names_numerical, outputCol="features")


Create the decision tree model from the train test set. Create the pipeline.

In [11]:
# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol=target_var_index, featuresCol="features", minInfoGain=0.05, maxBins=80)

pipeline = Pipeline(stages=[stringIndexer, assembler, dt])

# Fit the pipeline to training documents.
ctModel = pipeline.fit(df_train)

Take a look at our indexed target classes.

In [13]:
dist_coll_sev = df_train.groupBy("collision_severity").count().toPandas()
dist_coll_sev

,collision_severity,count
0,property damage only,46007
1,pain,18391
2,other injury,9206
3,fatal,496
4,severe injury,1690


Store default proportion (majority class frequency), for performance comparison later on.

In [15]:
default_p = dist_coll_sev.sort_values(["count"], ascending=False).iloc[0,1] / sum(dist_coll_sev["count"])
default_p

0.6070325900514579

The performance on the training set.

In [21]:
print(ctModel.stages[2].toDebugString)
df_preds = ctModel.transform(df_train)
accuracy = showPerformanceOnTest(df_preds, default_p)

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_ce1f344574b3, depth=4, numNodes=13, numClasses=5, numFeatures=54
  If (feature 38 <= 0.5)
   Predict: 0.0
  Else (feature 38 > 0.5)
   If (feature 46 <= 0.5)
    If (feature 45 <= 0.5)
     Predict: 1.0
    Else (feature 45 > 0.5)
     If (feature 37 <= 0.5)
      Predict: 3.0
     Else (feature 37 > 0.5)
      Predict: 4.0
   Else (feature 46 > 0.5)
    If (feature 45 <= 0.5)
     Predict: 2.0
    Else (feature 45 > 0.5)
     If (feature 37 <= 0.5)
      Predict: 3.0
     Else (feature 37 > 0.5)
      Predict: 4.0

Accuracy: 0.9946562871091172
Huberty Index: 0.9864016385186178


Variables used by the decision tree:

In [22]:
a= indexOutputCols+col_names_numerical
[a[38], a[37], a[45], a[46]]

['injured_victims',
 'killed_victims',
 'severe_injury_count',
 'other_visible_injury_count']

Have a look at the performance metrics of the model on the test set.

In [25]:
dist_coll_sev = df_test.groupBy("collision_severity").count().toPandas()
default_p = dist_coll_sev.sort_values(["count"], ascending=False).iloc[0,1] / sum(dist_coll_sev["count"])
df_preds = ctModel.transform(df_test)
accuracy = showPerformanceOnTest(df_preds, default_p)

Accuracy: 0.9939012394255361
Huberty Index: 0.9846895509845212


In [ ]:
# Save memory
del df, df_test, df_train, df_preds

Now let's try with the whole dataset.

In [28]:
# read the dataset:
df = sqlContext.read.format('jdbc').\
     options(url='jdbc:sqlite:switrs.sqlite',\
     dbtable='collisions',driver='org.sqlite.JDBC').load()

Let's get just 40% of the dataset, to try our model on.

In [29]:
df_half = df.sample(0.4, 42)
df_half.count()

3671398

In [30]:
df = df_half
del df_half
df.cache()

case_id,jurisdiction,officer_id,reporting_district,chp_shift,population,county_city_location,special_condition,beat_type,chp_beat_type,city_division_lapd,chp_beat_class,beat_number,primary_road,secondary_road,distance,direction,intersection,weather_1,weather_2,state_highway_indicator,caltrans_county,caltrans_district,state_route,route_suffix,postmile_prefix,postmile,location_type,ramp_intersection,side_of_highway,tow_away,collision_severity,killed_victims,injured_victims,party_count,primary_collision_factor,pcf_violation_code,pcf_violation_category,pcf_violation,pcf_violation_subsection,hit_and_run,type_of_collision,motor_vehicle_involved_with,pedestrian_action,road_surface,road_condition_1,road_condition_2,lighting,control_device,chp_road_type,pedestrian_collision,bicycle_collision,motorcycle_collision,truck_collision,not_private_property,alcohol_involved,statewide_vehicle_type_at_fault,chp_vehicle_type_at_fault,severe_injury_count,other_visible_injury_count,complaint_of_pain_injury_count,pedestrian_killed_count,pedestrian_injured_count,bicyclist_killed_count,bicyclist_injured_count,motorcyclist_killed_count,motorcyclist_injured_count,primary_ramp,secondary_ramp,latitude,longitude,collision_date,collision_time,process_date
3495044,9540,17076,null,1,9,1900,0,1,interstate,null,chp other,506,RT 5,RT 126,1000.0,south,0,clear,null,1,LA,7,5,null,R,55.29,highway,null,northbound,0,property damage only,0,0,2,vehicle code viol...,null,speeding,22350,null,not hit and run,rear end,other motor vehicle,no pedestrian inv...,dry,construction,null,daylight,none,1,null,null,null,1,1,null,truck or truck tr...,26,0,0,0,0,0,0,0,0,0,null,null,null,null,2008-01-02,11:30:00,2008-11-10
3511130,9480,9680,null,1,9,5400,0,3,county road area,null,chp other,006,ROAD 188,AVENUE 242,1320.0,south,0,cloudy,null,0,null,null,null,null,null,null,null,null,null,1,property damage only,0,0,2,vehicle code viol...,null,improper turning,22107,null,not hit and run,broadside,other motor vehicle,no pedestrian inv...,wet,normal,null,dark with no stre...,none,0,null,null,null,1,1,null,truck or truck tr...,27,0,0,0,0,0,0,0,0,0,null,null,null,null,2008-01-09,06:30:00,2008-08-08
3511299,109,8284,1,5,7,0109,0,0,not chp,null,not chp,06X,WEST MACARTHUR BL,MARKET ST,0.0,null,1,raining,null,0,null,null,null,null,null,null,null,null,null,1,property damage only,0,0,2,vehicle code viol...,null,automobile right ...,21800,D,not hit and run,broadside,other motor vehicle,no pedestrian inv...,wet,normal,null,daylight,not functioning,0,null,null,null,null,1,null,passenger car,01,0,0,0,0,0,0,0,0,0,null,null,null,null,2008-01-04,13:51:00,2008-08-08
3511310,1900,489624,2611,5,6,1950,0,0,not chp,null,not chp,262T1,PALMDALE BL,RT 14,0.0,null,1,clear,null,1,LA,7,138,null,null,43.531,intersection,5,eastbound,1,other injury,0,1,2,unknown,null,unknown,null,null,not hit and run,head-on,other motor vehicle,no pedestrian inv...,dry,normal,null,daylight,functioning,0,null,null,null,1,1,null,null,null,0,1,0,0,0,0,0,0,0,null,NF,null,null,2008-01-14,07:30:00,2008-10-25
3511315,9655,009175,null,1,9,3300,0,3,county road area,null,chp other,002,BONITA AV,BROADWAY ST,5.0,east,0,cloudy,null,0,null,null,null,null,null,null,null,null,null,1,property damage only,0,0,1,vehicle code viol...,null,speeding,22350,null,not hit and run,overturned,non-collision,no pedestrian inv...,dry,normal,null,daylight,none,0,null,null,null,1,1,null,truck or truck tr...,79,0,0,0,0,0,0,0,0,0,null,null,null,null,2008-01-08,08:41:00,2008-08-11
3511330,9755,017269,null,3,9,4200,0,1,us highway,null,chp other,004,RT 101,NOJOQUI CREEK BRI...,50.0,north,0,clear,null,1,SB,5,101,null,null,56.04,highway,null,southbound,1,property damage only,0,0,1,vehicle code viol...,null,improper turning,22107,null,not hit and run,hit object,fixed object,no pedestrian inv...,dry,normal,null,dark with no stre...,none,1,null,null,null,1,1,null,truck or truck tr...,26,0,0,0,0,0,0,0,0,0,null,null,null,null,2008-01-10,00:35:00,2008-08-20
3511331,3902,4800,null,5,5

Clean up the dataset.

In [31]:
# clean dataset
col_names = df.columns
col_names_categorical = [col for (col, dataType) in df.dtypes if dataType == 'string']
col_names_numeric = [col for col in col_names if col not in col_names_categorical]   

# remove columns that have no interesting information for our purpose
remove_cols = ["case_id", "process_date", "officer_id"]
   
# Columns to eliminate because they containvery few rows with data
remove_cols += ["secondary_ramp", "primary_ramp", "latitude", "longitude", "reporting_district"]

# categorical feature with too many different values
remove_cols += ["primary_road", "secondary_road", "collision_date", "postmile", 
                      "beat_number", "collision_time","county_city_location", 
                      "jurisdiction", "state_route", "pcf_violation"]
# no variability column
remove_cols += ["not_private_property"]

for col in remove_cols:
    if col in col_names_categorical: 
        col_names_categorical.remove(col)
    elif col in col_names_numeric:
        col_names_numeric.remove(col)
    if col in col_names:
        col_names.remove(col)
    df = df.drop(col)

from pyspark.sql.types import StringType
# Numerical variables that should be categorical
#   jurisdiction
#   caltrans_district
#   ramp_intersection
#   state_route
#   pcf_violation
#   postmile
df = df.withColumn("caltrans_district", df['caltrans_district'].cast(StringType()))\
    .withColumn("ramp_intersection", df['ramp_intersection'].cast(StringType()))

if "caltrans_district" not in col_names_categorical:
    col_names_categorical.append("caltrans_district")
    col_names_numeric.remove("caltrans_district")
if "ramp_intersection" not in col_names_categorical:
    col_names_categorical.append("ramp_intersection")
    col_names_numeric.remove("ramp_intersection")

# for these columns, missings are 0 (they only have 1s)
col_names_na_to_zero = ["pedestrian_collision", "bicycle_collision", "motorcycle_collision", "truck_collision", "alcohol_involved"]
df = df.fillna(0, subset=col_names_na_to_zero)
# set missing to mode (zero)
df = df.fillna(0, subset="killed_victims")
df = df.fillna(0, subset="injured_victims")
# set missing to a special category '2'
missing_binary_cols = ["intersection", "state_highway_indicator", "tow_away"]
df = df.fillna(2, subset=missing_binary_cols)

# "Not Stated" for other categorical columns
df = df.fillna("Not Stated", subset=col_names_categorical)

Looking if any missing values persist. We drop missing values' rows if they are not more then 0.5% of total rows.

In [32]:
if ((df.count() - df.dropna().count()) / df.count()) > 0.005:
    raise Exception("More then 0.5% of missing values. Algorithm stops here")
else:
    df = df.dropna()

Take a look at our indexed target classes.

In [33]:
dist_coll_sev = df.groupBy("collision_severity").count().toPandas()
dist_coll_sev

,collision_severity,count
0,property damage only,2223554
1,M,1
2,pain,885900
3,other injury,451189
4,fatal,25550
5,Not Stated,9
6,severe injury,85192


We calculate the default proportion (majority class frequency) to later calculate the Huberty Index.

In [34]:
# store default proportion (majority class frequency)
line_total = sum(dist_coll_sev["count"])
default_p = dist_coll_sev.sort_values(["count"], ascending=False).iloc[0,1] / line_total
default_p

0.6056428142436322

Lets build the features and apply the model.

In [36]:
# Now apply the model
df_preds = ctModel.transform(df)

In [37]:
accuracy = showPerformanceOnTest(df_preds, default_p, line_total)
accuracy

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1211, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: An error occurred while calling o605.count